导入库

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from transformers import BertModel, BertTokenizer
import timm
import numpy as np

选用图像编码器，采用 ViT

In [ ]:
class ViT(nn.Module):
    def __init__(self, output_dim):
        super(ViT, self).__init__()
        self.model = timm.create_model('vit_base_patch16_224', pretrained=True)

    def forward(self, x):
        x = self.model(x)
        return x

选用文本编码器，采用 BERT

In [ ]:
class TextEncoder(nn.Module):
    def __init__(self):
        super(TextEncoder, self).__init__()
        BERT_LOCAL_PATH = './bert-base-uncased'
        self.model = BertModel.from_pretrained(BERT_LOCAL_PATH)
        self.tokenizer = BertTokenizer.from_pretrained(BERT_LOCAL_PATH)
    
    def forward(self, text):
        """
        return_tensors='pt'指定返回的是 PyTorch 张量。padding=True 和 truncation=True 表示如果输入的文本长度不一致 
        将进行填充或截断 以确保所有文本具有相同的长度 这是 BERT 模型处理批量数据时的要求。
        """
        encoded_input = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**encoded_input)
        # 取[CLS]标记的输出作为句子的表示
        return outputs.last_hidden_state[:, 0, :]

构建 CLIP 模型

In [ ]:
class CLIP(nn.Module):
    def __init__(self, image_output_dim, text_output_dim):
        super(CLIP, self).__init__()
        self.image_encoder = ViT(image_output_dim)
        self.text_encoder = TextEncoder()

        # 因为图像和文本 emb 可能维度不同(图像 512, 文本 768)所以需要对图像和文本的 emb 再经过一层以将维度持平
        self.W_i = nn.Parameter(torch.randn(image_output_dim, text_output_dim))
        self.W_t = nn.Parameter(torch.randn(768, text_output_dim)) # BERT-base 的最后隐藏层大小为 768

    def forward(self, image, text):
        image_emb = self.image_encoder(image) # （b, 3, 224, 224） -> （b, 512）
        text_emb = self.text_encoder(text)  # (b) -> (b, 768)

        # 将图像和文本的 emb 映射到相同的维度
        image_emb = torch.matmul(image_emb, self.W_i)   # (b, 512)
        text_emb = torch.matmul(text_emb, self.W_t) # (b, 512)

        # 计算余弦相似度
        logits = torch.matmul(image_emb, text_emb.T)    # (b, b)

        return logits

加载 CIFAR10 数据集

In [ ]:
def load_cifar10_dataset():
    # 调整图像大小, 转换为 PyTorch 张量并将像素值归一化到 [0, 1] 的范围。
    transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
    train_dataset = CIFAR10(root='./cifar10', train=True, download=True, transform=transform)
    loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    classes = train_dataset.classes
    return loader, classes

主函数

In [ ]:
def main():
    # 加载数据集
    dataset, classes = load_cifar10_dataset()
    print(f"Classes: {classes}")
    # 初始化 CLIP 模型
    clip_model = CLIP(image_output_dim=512, text_output_dim=512)

    for images, labels in dataset:
        # 获得一个 batch 的图像和标签
        texts = [classes[label] for label in labels]
        logits = clip_model(images, texts)
        print(f"Logits shape: {logits.shape}")
        # 对角线是真实的值, 故把位置当作真实标签
        labels = torch.arange(logits.shape[0])  # (0, 1, 2, 3)

        # 计算损失 loss_i 是每一张图像我都要把它判定为正确得文本，而 loss_t 是每一个文本我都要把它判定为正确得图像
        # logits 是模型的输出, 假设形状为(b, c) b 是批次大小,c 是类别数量; 该函数, 表示的是每个批次对应各个类别的预测分数. 
        # 在这里即各个 Text/Image 对应各个 Image/Text 的相似度
        # labels 通常是一个一维张量, 包含每个样本的真实标签
        loss_i = torch.nn.CrossEntropyLoss()(logits, labels)
        loss_t = torch.nn.CrossEntropyLoss()(logits.T, labels)

        loss = (loss_i + loss_t) / 2
        print(f"Loss: {loss}")